In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
import random

plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['figure.dpi'] = 100


In [ ]:
### Schelling Segregation Model Implementation


In [ ]:
def initialize_grid(size=50, occupancy_rate=0.9):
    """
    Initialize a grid with two types of agents (1 and 2) randomly placed.
    Empty cells are represented by 0.
    
    Parameters:
    - size: grid size (size x size)
    - occupancy_rate: fraction of cells that are occupied (default 0.9)
    
    Returns:
    - grid: 2D numpy array with values 0 (empty), 1 (agent type X), 2 (agent type O)
    """
    total_cells = size * size
    occupied_cells = int(total_cells * occupancy_rate)
    
    # Create grid with zeros (empty)
    grid = np.zeros((size, size), dtype=int)
    
    # Randomly select positions for agents
    positions = np.random.choice(total_cells, occupied_cells, replace=False)
    
    # Half X agents, half O agents
    num_x = occupied_cells // 2
    num_o = occupied_cells - num_x
    
    # Place X agents (value 1)
    x_positions = positions[:num_x]
    for pos in x_positions:
        row = pos // size
        col = pos % size
        grid[row, col] = 1
    
    # Place O agents (value 2)
    o_positions = positions[num_x:]
    for pos in o_positions:
        row = pos // size
        col = pos % size
        grid[row, col] = 2
    
    return grid


In [ ]:
def get_neighbors(grid, row, col):
    """
    Get the 8 neighbors of a cell (including diagonals).
    
    Returns:
    - neighbors: list of values of neighboring cells
    """
    size = grid.shape[0]
    neighbors = []
    
    for dr in [-1, 0, 1]:
        for dc in [-1, 0, 1]:
            if dr == 0 and dc == 0:
                continue
            nr = row + dr
            nc = col + dc
            if 0 <= nr < size and 0 <= nc < size:
                neighbors.append(grid[nr, nc])
    
    return neighbors


In [ ]:
def is_satisfied(grid, row, col, R):
    """
    Check if an agent at (row, col) is satisfied.
    An agent is satisfied if at least R% of its neighbors are of the same type.
    
    Parameters:
    - grid: the current grid state
    - row, col: position of the agent
    - R: satisfaction threshold (0 to 1)
    
    Returns:
    - True if satisfied, False otherwise
    """
    agent_type = grid[row, col]
    
    # Empty cells are always "satisfied" (don't need to move)
    if agent_type == 0:
        return True
    
    neighbors = get_neighbors(grid, row, col)
    
    # Count non-empty neighbors
    non_empty_neighbors = [n for n in neighbors if n != 0]
    
    if len(non_empty_neighbors) == 0:
        return True  # No neighbors, so satisfied
    
    # Count neighbors of same type
    same_type_count = sum(1 for n in non_empty_neighbors if n == agent_type)
    
    # Check if at least R% are same type
    same_type_ratio = same_type_count / len(non_empty_neighbors)
    
    return same_type_ratio >= R


In [ ]:
def find_dissatisfied_agents(grid, R):
    """
    Find all dissatisfied agents in the grid.
    
    Returns:
    - dissatisfied: list of (row, col) tuples for dissatisfied agents
    """
    dissatisfied = []
    size = grid.shape[0]
    
    for row in range(size):
        for col in range(size):
            if grid[row, col] != 0:  # Only check occupied cells
                if not is_satisfied(grid, row, col, R):
                    dissatisfied.append((row, col))
    
    return dissatisfied


In [ ]:
def find_empty_cells(grid):
    """
    Find all empty cells in the grid.
    
    Returns:
    - empty: list of (row, col) tuples for empty cells
    """
    empty = []
    size = grid.shape[0]
    
    for row in range(size):
        for col in range(size):
            if grid[row, col] == 0:
                empty.append((row, col))
    
    return empty


In [ ]:
def move_agents(grid, dissatisfied, empty_cells):
    """
    Move dissatisfied agents to empty cells.
    Uses random assignment: shuffle both lists and match them.
    
    Parameters:
    - grid: current grid state
    - dissatisfied: list of (row, col) tuples for dissatisfied agents
    - empty_cells: list of (row, col) tuples for empty cells
    
    Returns:
    - new_grid: grid after moving agents
    """
    new_grid = grid.copy()
    
    # Shuffle both lists for random assignment
    dissatisfied_shuffled = dissatisfied.copy()
    empty_shuffled = empty_cells.copy()
    random.shuffle(dissatisfied_shuffled)
    random.shuffle(empty_shuffled)
    
    # Move agents to empty cells
    for i, (old_row, old_col) in enumerate(dissatisfied_shuffled):
        if i < len(empty_shuffled):
            new_row, new_col = empty_shuffled[i]
            # Move agent to new location
            new_grid[new_row, new_col] = grid[old_row, old_col]
            # Clear old location
            new_grid[old_row, old_col] = 0
    
    return new_grid


In [ ]:
def simulate_schelling(size=50, R=0.5, max_iterations=100, occupancy_rate=0.9):
    """
    Run the Schelling segregation simulation.
    
    Parameters:
    - size: grid size
    - R: satisfaction threshold (0 to 1)
    - max_iterations: maximum number of iterations
    - occupancy_rate: fraction of cells occupied
    
    Returns:
    - history: list of grid states at each iteration
    - dissatisfied_counts: list of number of dissatisfied agents at each iteration
    """
    grid = initialize_grid(size, occupancy_rate)
    history = [grid.copy()]
    dissatisfied_counts = []
    
    for iteration in range(max_iterations):
        # Find dissatisfied agents
        dissatisfied = find_dissatisfied_agents(grid, R)
        num_dissatisfied = len(dissatisfied)
        dissatisfied_counts.append(num_dissatisfied)
        
        # If no one is dissatisfied, we've reached equilibrium
        if num_dissatisfied == 0:
            print(f"Equilibrium reached at iteration {iteration} for R={R}")
            break
        
        # Find empty cells (including cells that will become empty)
        empty_cells = find_empty_cells(grid)
        
        # Add dissatisfied cells to the market (they become empty)
        empty_cells.extend(dissatisfied)
        
        # Move dissatisfied agents to empty cells
        grid = move_agents(grid, dissatisfied, empty_cells)
        
        # Record state
        history.append(grid.copy())
    
    return history, dissatisfied_counts


In [ ]:
### Step 1: Initialize and visualize the initial random map


In [ ]:
# Initialize a random grid
grid = initialize_grid(size=50, occupancy_rate=0.9)

# Create colormap: white (empty), red (X agents), blue (O agents)
colors = ['white', 'red', 'blue']
cmap = ListedColormap(colors)

plt.figure(figsize=(10, 10))
plt.imshow(grid, cmap=cmap, vmin=0, vmax=2)
plt.title('Initial Random Grid (Half X/Red, Half O/Blue)')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Grid size: {grid.shape}")
print(f"X agents (red): {np.sum(grid == 1)}")
print(f"O agents (blue): {np.sum(grid == 2)}")
print(f"Empty cells (white): {np.sum(grid == 0)}")


In [ ]:
### Step 2: Run simulations for all 9 R values and create GIFs


In [ ]:
# Define the 9 R values
R_values = [0, 1/8, 1/4, 3/8, 1/2, 5/8, 3/4, 7/8, 1]

# Store results for all R values
all_histories = {}
all_dissatisfied_counts = {}

# Colors for visualization
colors = ['white', 'red', 'blue']
cmap = ListedColormap(colors)

print("Running simulations for all R values...")
for R in R_values:
    print(f"\nRunning simulation for R = {R:.3f}...")
    history, dissatisfied_counts = simulate_schelling(size=50, R=R, max_iterations=200, occupancy_rate=0.9)
    all_histories[R] = history
    all_dissatisfied_counts[R] = dissatisfied_counts
    print(f"  Completed {len(history)} iterations")
    print(f"  Final dissatisfied agents: {dissatisfied_counts[-1] if dissatisfied_counts else 0}")


In [ ]:
# Create GIFs for each R value
print("\nCreating GIFs...")

# Create filename-friendly R labels
R_labels = {
    0: '0',
    1/8: '0_125',
    1/4: '0_25',
    3/8: '0_375',
    1/2: '0_5',
    5/8: '0_625',
    3/4: '0_75',
    7/8: '0_875',
    1: '1'
}

for R in R_values:
    history = all_histories[R]
    
    # Limit to reasonable number of frames for GIF
    max_frames = 100
    if len(history) > max_frames:
        # Sample frames evenly
        indices = np.linspace(0, len(history)-1, max_frames, dtype=int)
        history_sampled = [history[i] for i in indices]
    else:
        history_sampled = history
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    im = ax.imshow(history_sampled[0], cmap=cmap, vmin=0, vmax=2)
    ax.set_title(f'Schelling Segregation Model - R = {R:.3f} - Iteration 0', fontsize=14)
    ax.axis('off')
    
    def animate(frame):
        ax.clear()
        im = ax.imshow(history_sampled[frame], cmap=cmap, vmin=0, vmax=2)
        ax.set_title(f'Schelling Segregation Model - R = {R:.3f} - Iteration {frame}', fontsize=14)
        ax.axis('off')
        return [im]
    
    anim = FuncAnimation(fig, animate, frames=len(history_sampled), interval=100, blit=True, repeat=True)
    
    filename = f'schelling_R_{R_labels[R]}.gif'
    anim.save(filename, writer='pillow', fps=10)
    print(f"  Saved {filename}")
    plt.close(fig)

print("\nAll GIFs created!")


In [ ]:
### Step 3: Plot number of dissatisfied households vs time for all R values


In [ ]:
plt.figure(figsize=(12, 8))

# Plot curves for each R value
for R in R_values:
    dissatisfied_counts = all_dissatisfied_counts[R]
    iterations = range(len(dissatisfied_counts))
    plt.plot(iterations, dissatisfied_counts, label=f'R = {R:.3f}', linewidth=2)

plt.xlabel('Iteration (Time)', fontsize=14)
plt.ylabel('Number of Dissatisfied Households', fontsize=14)
plt.title('Number of Dissatisfied Households vs Time for Different R Values', fontsize=16)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
